In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import matplotlib.pyplot as plt
import torchtext
import spacy
import tiktoken
import pandas as pd
import numpy as np
from gensim.corpora.dictionary import Dictionary

from tqdm.notebook import tqdm

In [ ]:
#GLOBALS

block_size = 256 #This is the value of T

In [2]:
import os
en = spacy.load('en_core_web_sm')
de = spacy.load('de_core_news_sm')
#spacy.load('en_core_web_sm')
#spacy.load('de_core_news_sm')

#dataset https://nlp.stanford.edu/projects/nmt/

In [ ]:
def create_dataset():
    vocab_en = Dictionary()

    # vocab_en.add_documents([["<START>"]])
    # vocab_en.add_documents([["<END>"]])
    vocab_en.add_documents([["<PAD>"]])

    vocab_ger = Dictionary()

    vocab_ger.add_documents([["<START>"]])
    vocab_ger.add_documents([["<END>"]])
    vocab_ger.add_documents([["<PAD>"]])

    en_max = 0 
    with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
        idx_en = []
        sentences_en = []
        for i in tqdm(range(400000)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            doc = en(line)
            sentence = [[token.text.lower() for token in doc]]
            if len(sentence[0]) > en_max:
                en_max = len(sentence[0])
                print(en_max)
            vocab_en.add_documents(sentence)
            sentence = ["<PAD>"] + sentence[0] + ["<PAD>"]

            if len(sentence) <= block_size:
                len_pad = block_size - len(sentence)
                sentence = sentence + len_pad*"<PAD> ".split()
                assert len(sentence) == block_size, print(len(sentence))
                idx_en.append(i)
                sentences_en.append(sentence)

    print(en_max)    
    print(len(sentences_en))
    print(len(vocab_en.token2id))

    vocab_en.save(os.getcwd()+"\\vocab\\vocab_en.pkl")


    de_max = 0 
    with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
        idx_de = []
        sentences_de = []
        for i in tqdm(range(400000)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            doc = de(line)
            sentence = [[token.text.lower() for token in doc]]
            if len(sentence[0]) > de_max:
                de_max = len(sentence[0])
                print(de_max)
            vocab_ger.add_documents(sentence)
            sentence = ["<START>"] + sentence[0] + ["<END>"]
            if len(sentence) <= block_size:
                len_pad = block_size - len(sentence)
                sentence = sentence + len_pad*"<PAD> ".split()
                assert len(sentence) == block_size, print(len(sentence))
                idx_de.append(i)
                sentences_de.append(sentence)

    print(de_max)  
    print(len(sentences_de))
    print(len(vocab_ger.token2id))

    vocab_ger.save(os.getcwd()+"\\vocab\\vocab_ger.pkl")

    df = pd.DataFrame()

    df['idx_en'] = idx_en
    df['inputs'] = sentences_en
    df['idx_targets'] = idx_de
    df['targets'] = sentences_de

    df.to_csv(os.getcwd()+"\\data\\data.csv", index = False)

In [ ]:
create = False
if create:
    create_dataset
else:
    vocab_en = Dictionary.load(os.getcwd()+"\\vocab\\vocab_en.pkl")
    vocab_ger = Dictionary.load(os.getcwd()+"\\vocab\\vocab_ger.pkl")

In [ ]:
#TRAIN AND VAL DATASETS

df = pd.read_csv("")

In [ ]:
#BATCH LOADER

rough work

In [3]:
test = "hello my name is hello name is my"
test2 = "one two three four"

doc_ = en(test)

for token in doc_:
    print(token.text.lower())
sentence_ = [[token.text for token in doc_]]
#sentence = [sentence + ['one','two']]

# sentence

sentence2 = [['one', 'two', 'hello']]

hello
my
name
is
hello
name
is
my


In [4]:
vocab = Dictionary()

vocab.add_documents(sentence_)

In [8]:
vocab[0]

'hello'

In [10]:
vocab.doc2idx(['hello'])

[0]

In [12]:
vocab.save(os.getcwd()+"\\vocab\\vocab.pkl")

In [15]:
vocab_ = Dictionary.load(os.getcwd()+"\\vocab\\vocab.pkl")

vocab_[0]
vocab_.doc2idx(['hello'])

[0]

In [16]:
df = pd.DataFrame()

In [20]:
vocab_en = Dictionary()

# vocab_en.add_documents([["<START>"]])
# vocab_en.add_documents([["<END>"]])
vocab_en.add_documents([["<PAD>"]])

vocab_ger = Dictionary()

vocab_ger.add_documents([["<START>"]])
vocab_ger.add_documents([["<END>"]])
vocab_ger.add_documents([["<PAD>"]])

max in german text is 238
max in english text is 342

pad both to 128

15 minutes for 100000 sentences, both german and english

328514

In [21]:
en_max = 0 
with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
    idx_en = []
    sentences_en = []
    for i in tqdm(range(400000)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        doc = en(line)
        sentence = [[token.text.lower() for token in doc]]
        if len(sentence[0]) > en_max:
            en_max = len(sentence[0])
            print(en_max)
        vocab_en.add_documents(sentence)
        sentence = ["<PAD>"] + sentence[0] + ["<PAD>"]
        
        idx_en.append(i)
        if len(sentence) <= 256:
            len_pad = 256 - len(sentence)
            sentence = sentence + len_pad*"<PAD> ".split()
            assert len(sentence) == 256, print(len(sentence))
            sentences_en.append(sentence)
        else:
            sentences_en.append(np.NaN)

print(en_max)    
print(len(sentences_en))
print(len(vocab_en.token2id))

vocab_en.save(os.getcwd()+"\\vocab\\vocab_en.pkl")

  0%|          | 0/400000 [00:00<?, ?it/s]

32
34
47
49
54
57
58
67
87
91
92
97
98
100
105
117
130
153
169
182
244
245
249
342
342
399999
176008


In [22]:
de_max = 0 
with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
    idx_de = []
    sentences_de = []
    for i in tqdm(range(400000)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        doc = de(line)
        sentence = [[token.text.lower() for token in doc]]
        if len(sentence[0]) > de_max:
            de_max = len(sentence[0])
            print(de_max)
        vocab_ger.add_documents(sentence)
        sentence = ["<START>"] + sentence[0] + ["<END>"]

        idx_de.append(i)
        if len(sentence) < 256:
            len_pad = 256 - len(sentence)
            sentence = sentence + len_pad*"<PAD> ".split()
            assert len(sentence) == 256, print(len(sentence))
            sentences_de.append(sentence)
        else:
            sentences_de.append(np.NaN)

print(de_max)  
print(len(sentences_de))
print(len(vocab_ger.token2id))

vocab_ger.save(os.getcwd()+"\\vocab\\vocab_ger.pkl")

  0%|          | 0/400000 [00:00<?, ?it/s]

34
43
48
63
64
71
75
86
98
100
106
135
152
238
238
400000
332799


In [ ]:
assert idx_en == idx_de, print("sentences do not align")

In [24]:
df['idx_en'] = pd.Series(idx_en)
df['inputs'] = pd.Series(sentences_en)
df['idx_targets'] = pd.Series(idx_de)
df['targets'] = pd.Series(sentences_de)

In [25]:
df.tail()

,idx_en,inputs,idx_targets,targets
399994,399995,"[<PAD>, the, world, of, warcraft, character, c...",399994,"[<START>, in, der, veranstaltung, zu, kunst, u..."
399995,399996,"[<PAD>, blizzard, developers, tom, chilton, an...",399995,"[<START>, die, world, of, warcraft, klassenver..."
399996,399997,"[<PAD>, in, their, forty, #, #, at##-##at, #, ...",399996,"[<START>, blizzards, entwickler, tom, chilton,..."
399997,399998,"[<PAD>, beyond, the, panel, discussions, ,, ma...",399997,"[<START>, in, ihrem, 45, #, #, at##-##at, #, #..."
399998,399999,"[<PAD>, at, the, time, of, this, report, ,, ma...",399998,"[<START>, neben, den, veranstaltungen, schaute..."


In [26]:
df2 = df.loc[df['idx_en'] == df['idx_targets']]

In [27]:
df2

,idx_en,inputs,idx_targets,targets
0,0,"[<PAD>, iron, cement, is, a, ready, for, use, ...",0,"[<START>, iron, cement, ist, eine, gebrauchs, ..."
1,1,"[<PAD>, iron, cement, protects, the, ingot, ag...",1,"[<START>, nach, der, aushärtung, schützt, iron..."
2,2,"[<PAD>, a, fire, restant, repair, cement, for,...",2,"[<START>, feuerfester, reparaturkitt, für, feu..."
3,3,"[<PAD>, construction, and, repair, of, highway...",3,"[<START>, der, bau, und, die, reparatur, der, ..."
4,4,"[<PAD>, an, announcement, must, be, commercial...",4,"[<START>, die, mitteilungen, sollen, den, gesc..."
...,...,...,...,...
328509,328509,"[<PAD>, description, :, historical, building, ...",328509,"[<START>, beschreibung, :, prague, central, re..."
328510,328510,"[<PAD>, description, :, centrally, situated, o...",328510,"[<START>, beschreibung, :, zentral, gelegen, ,..."
328511,328511,"[<PAD>, description, :, tucked, into, the, tur...",328511,"[<START>, beschreibung, :, umgeben, von, einer..."
328512,328512,"[<PAD>, description, :, easter, ,, christmas, ...",328512,"[<START>, ostern, ,, weihnachten, und, neues, ..."


In [ ]:
len(df)

399999

In [ ]:
df.to_csv(os.getcwd()+"\\data\\data.csv", index = False)

In [19]:
df['inputs'].values[0]

['<PAD>',
 'iron',
 'cement',
 'is',
 'a',
 'ready',
 'for',
 'use',
 'paste',
 'which',
 'is',
 'laid',
 'as',
 'a',
 'fillet',
 'by',
 'putty',
 'knife',
 'or',
 'finger',
 'in',
 'the',
 'mould',
 'edges',
 '(',
 'corners',
 ')',
 'of',
 'the',
 'steel',
 'ingot',
 'mould',
 '.',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD

In [20]:
vocab_en.doc2idx(df['inputs'].values[0])

[0,
 15,
 7,
 16,
 4,
 24,
 12,
 27,
 22,
 28,
 16,
 18,
 5,
 4,
 10,
 6,
 23,
 17,
 21,
 11,
 13,
 26,
 19,
 9,
 1,
 8,
 2,
 20,
 26,
 25,
 14,
 19,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0